In [1]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
from elasticsearch_dsl import Search
from datetime import datetime
from tqdm import tqdm
import json

In [4]:
es = Elasticsearch(
    hosts = [{'host': 'localhost', 'port': 9200}],
    connection_class = RequestsHttpConnection,
)
s = Search(using=es, index='twitter_streaming')
with open('snapshot.jsonl', 'w') as fopen:
    
    for hit in tqdm(s.scan()):
        h = hit.to_dict()
        fopen.write(f'{json.dumps(h)}\n')

7075025it [17:34, 6709.84it/s]


In [5]:
!wc -l snapshot.jsonl

7075025 snapshot.jsonl


In [6]:
!head -n 1 snapshot.jsonl

{"datetime": "2022-04-18T05:57:04", "datetime_gmt8": "2022-04-18T13:57:04", "data_text": "kekal halal kak https://t.co/YHKqszqPnS", "body": "kekal halal kak https://t.co/YHKqszqPnS", "screen_name": "Luke_Sebastian2", "followers_count": 10413, "friends_count": 72, "listed_count": 6, "favourites_count": 1494, "statuses_count": 948, "quoted_status_text": "NULL", "lang": "in", "retweet": "false", "retweet_text": "NULL", "retweet_text_full": "NULL", "retweet_count": 0, "retweet_detail": {}, "quote_count": 0, "favorite_count": 0, "reply_count": 0, "id_str": "1515932406368202753", "tweet": {"created_at": "Mon Apr 18 05:57:04 +0000 2022", "id": 1515932406368202753, "id_str": "1515932406368202753", "text": "kekal halal kak\ud83d\ude0f\ud83e\udd2b https://t.co/YHKqszqPnS", "display_text_range": [0, 17], "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "truncated": false, "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_rep

In [11]:
es = Elasticsearch(
    hosts = [{'host': 'localhost', 'port': 9200}],
    connection_class = RequestsHttpConnection,
)
s = Search(using=es, index='twitter_streaming')
s.aggs.bucket('min', 'min', field='datetime_gmt8')
s = s[:0]
s.execute().to_dict()

{'took': 13,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'min': {'value': 1650213007000.0,
   'value_as_string': '2022-04-17T16:30:07.000Z'}}}

In [12]:
es = Elasticsearch(
    hosts = [{'host': 'localhost', 'port': 9200}],
    connection_class = RequestsHttpConnection,
)
s = Search(using=es, index='twitter_streaming')
s.aggs.bucket('min', 'max', field='datetime_gmt8')
s = s[:0]
s.execute().to_dict()

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'min': {'value': 1662197032000.0,
   'value_as_string': '2022-09-03T09:23:52.000Z'}}}

In [13]:
!split -l 500000 -d --additional-suffix=.splitted snapshot.jsonl snapshot.jsonl